## Regression Analysis on Ionic Emissions Supply/Borrow



## Imports

In [ ]:
pip install dune-client pycoingecko

## Load data




In [ ]:
from dune_client.client import DuneClient
from google.colab import userdata
userdata.get('DUNE_API_KEY')



dune_api_key = userdata.get('DUNE_API_KEY')
dune = DuneClient(dune_api_key)
deposits = dune.get_latest_result_dataframe(4003406) # mrwild
print("deposits loaded")
borrows= dune.get_latest_result_dataframe(4301345) # mrwild
print("borrows loaded")
withdrawals = dune.get_latest_result_dataframe(4003238) # mrwild
print("withdrawals loaded")
tvl_agg = dune.get_latest_result_dataframe(4001052) # mrwild TVL by POOL
print("tvl by pool loaded")
protocol_tvl = dune.get_latest_result_dataframe(4301363) # NEW  TVL cumulative (old 4069195)
print("total tvl loaded")
protocol_tvl_notional = dune.get_latest_result_dataframe(4309385) # NEW TVL cumulative value (UZL)
print("protocol_tvl_notional loaded")


deposits loaded
borrows loaded
withdrawals loaded
tvl by pool loaded
total tvl loaded
protocol_tvl_notional loaded


In [ ]:
# copies of the dataframes
deposits_copy = deposits.copy()
borrows_copy = borrows.copy()
withdrawals_copy = withdrawals.copy()
tvl_agg_copy = tvl_agg.copy()
protocol_tvl_copy = protocol_tvl.copy()
protocol_tvl_notional_copy = protocol_tvl_notional.copy()


In [ ]:
deposits_copy.tail(5)

,date,vaultName,mintedAmount_USD
4956,2024-12-09 00:00:00.000 UTC,ionwUSDMb,1109.592419
4957,2024-12-09 00:00:00.000 UTC,ionweETH,0.578915
4958,2024-12-09 00:00:00.000 UTC,ionweETHb,0.400000
4959,2024-12-09 00:00:00.000 UTC,ionwrsETHm,43.401093
4960,2024-12-09 00:00:00.000 UTC,ionwsuperOETHb,0.000612


In [ ]:
borrows_copy.head(5)

,date,vaultName,daily_borrowed_amount_usd,total_borrowed_in_vault_usd
0,2024-12-09,ionAEROb,0.0,181433.45749671175
1,2024-12-09,ionEURCb,0.0,40918.38750970053
2,2024-12-09,ionLUSDo,0.0,4.099560122502437
3,2024-12-09,ionMBTCm,0.0,674765.704354458
4,2024-12-09,ionMODEmi,0.0,541561.6161029352


In [ ]:
withdrawals_copy.tail(5)

,date,vaultName,redeemedAmount_USD
5011,2024-12-09 00:00:00.000 UTC,ionuSUIb,200.148949
5012,2024-12-09 00:00:00.000 UTC,ionweETHm,0.037433
5013,2024-12-09 00:00:00.000 UTC,ionwrsETHm,19.293805
5014,2024-12-09 00:00:00.000 UTC,ionwstETHb,0.142438
5015,2024-12-09 00:00:00.000 UTC,ionwsuperOETHb,1.000612


In [ ]:
tvl_agg_copy.tail(5)

,vaultName,TVL_USD,TotalBorrowed_USD,ActiveDeposits_USD
44,ionhyUSD,64926.378310,2.865935e+05,3.515199e+05
45,ionAEROb,133854.657239,1.728404e+05,3.066951e+05
46,ionWETHb,653273.152469,1.092938e+06,1.746211e+06
47,ioneUSDb,202224.552486,9.088363e+05,1.111061e+06
48,ionOGNb,6324.770903,0.000000e+00,6.324771e+03


In [ ]:
protocol_tvl_copy.tail(5)

,date,ionicVault,TVL,chain
16851,2024-12-09,ionweETHm,8.379128e+05,mode
16852,2024-12-09,ionwrsETHm,8.253815e+05,mode
16853,2024-12-09,ionwstETHb,6.571745e+04,base
16854,2024-12-09,ionwstETHo,2.717326e+01,optimism
16855,2024-12-09,ionwsuperOETHb,1.469007e+06,base


In [ ]:
protocol_tvl_notional_copy.tail(5)

,date,ionicVault,TVL,chain
16851,2024-12-09,ionweETHm,199.514424,mode
16852,2024-12-09,ionwrsETHm,196.530601,mode
16853,2024-12-09,ionwstETHb,13.909986,base
16854,2024-12-09,ionwstETHo,0.005752,optimism
16855,2024-12-09,ionwsuperOETHb,368.839371,base


## Data Preprocessing

In [ ]:
# from 11-15-12-15, take the emissions in row V-AC and multiply by how much time has passed thru the epoch,
# then we have all emissions per vault, per side (supply or borrow)
# size of pool, type of pool, and then model the impact of emissions based on those

In [ ]:
import pandas as pd
import numpy as np
from google.colab import auth
import gspread
from google.auth import default

# Ensure we're authenticated
auth.authenticate_user()
creds, _ = default()
access = gspread.authorize(creds)

# Connect to the workbook
WORKBOOK_ID = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
SHEET_ID = '357844907'

wb = access.open_by_key(WORKBOOK_ID)
sheet = wb.get_worksheet_by_id(int(SHEET_ID))

# Get chain names, vault names, and emissions data for both epochs
chain_names = sheet.get_values('E:E')
vault_names = sheet.get_values('F:F')
previous_emissions = sheet.get_values('N:U')  # Previous epoch columns
current_emissions = sheet.get_values('V:AC')  # Current epoch columns

if previous_emissions and current_emissions and vault_names and chain_names:
    # Create names DataFrame first
    chain_series = pd.Series(chain_names[2:len(current_emissions)])
    vault_series = pd.Series(vault_names[2:len(current_emissions)])

    names_df = pd.DataFrame({
        'chain': chain_series,
        'vault': vault_series
    })

    def clean_value(val):
        if pd.isna(val) or val == '':
            return ''
        return str(val).replace('[', '').replace(']', '').replace("'", "").replace('"', '').strip()

    # Clean the values
    names_df['chain'] = names_df['chain'].apply(clean_value)
    names_df['vault'] = names_df['vault'].apply(clean_value)

    # Forward fill the chain names to handle merged cells
    names_df['chain'] = names_df['chain'].replace('', np.nan).fillna(method='ffill')

    def get_chain_suffix(chain):
        chain_map = {
            'Mode': 'm',
            'Base': 'b',
            'Optimism': 'o',
            'FRAXTAL': 'f',
            'BOB': 'bob'
        }
        return chain_map.get(chain.strip(), '')

    # Combine into proper vault name format
    names_df['standardized_vault'] = names_df.apply(
        lambda x: f"ion{x['vault']}{get_chain_suffix(x['chain'])}"
        if x['vault'] != ''
        else '', axis=1
    )

    # Create emissions DataFrames for both epochs
    previous_df = pd.DataFrame(previous_emissions[2:], columns=[
        'prev_borrow_emissions_tokens',
        'prev_borrow_emissions_usd',
        'prev_borrow_emissions_apr',
        'prev_borrow_net_apr',
        'prev_supply_emissions_tokens',
        'prev_supply_emissions_usd',
        'prev_supply_emissions_apr',
        'prev_supply_net_apr'
    ])

    current_df = pd.DataFrame(current_emissions[2:], columns=[
        'curr_borrow_emissions_tokens',
        'curr_borrow_emissions_usd',
        'curr_borrow_emissions_apr',
        'curr_borrow_net_apr',
        'curr_supply_emissions_tokens',
        'curr_supply_emissions_usd',
        'curr_supply_emissions_apr',
        'curr_supply_net_apr'
    ])

    # Add standardized vault names to both DataFrames
    previous_df.insert(0, 'Vault', names_df['standardized_vault'])
    current_df.insert(0, 'Vault', names_df['standardized_vault'])

    # Merge the two epochs' data
    emissions_df = previous_df.merge(current_df, on='Vault', how='outer')

    def clean_numeric(val):
        """Clean and convert values to numeric, handling special cases"""
        if pd.isna(val) or val == '' or val in ['#N/A', '#DIV/0!']:
            return 0
        if isinstance(val, str):
            val = val.replace('$', '').replace(',', '').strip()
            if '%' in val:
                val = val.replace('%', '')
                try:
                    return float(val) / 100
                except ValueError:
                    return 0
            try:
                return float(val)
            except ValueError:
                return 0
        return float(val) if val else 0

    # Clean numeric columns for both epochs
    numeric_cols = emissions_df.columns.drop('Vault')
    for col in numeric_cols:
        emissions_df[col] = emissions_df[col].apply(clean_numeric)

    # Extract chain information
    emissions_df['chain'] = emissions_df['Vault'].str.extract('([mbo])$').fillna('unknown')
    emissions_df['chain'] = emissions_df['chain'].map({'m': 'mode', 'b': 'base', 'o': 'optimism'})

    # Print debug info
    print("Sample of standardized vault names and emissions data:")
    print(emissions_df.head())
    print("\nColumns in emissions_df:")
    print(emissions_df.columns.tolist())
    print("\nNumber of vaults with emissions:")
    print(f"Previous epoch - Borrow: {(emissions_df['prev_borrow_emissions_tokens'] > 0).sum()}")
    print(f"Previous epoch - Supply: {(emissions_df['prev_supply_emissions_tokens'] > 0).sum()}")
    print(f"Current epoch - Borrow: {(emissions_df['curr_borrow_emissions_tokens'] > 0).sum()}")
    print(f"Current epoch - Supply: {(emissions_df['curr_supply_emissions_tokens'] > 0).sum()}")

else:
    print("No data found in the specified ranges")

Sample of standardized vault names and emissions data:
      Vault  prev_borrow_emissions_tokens  prev_borrow_emissions_usd  \
0  ionAEROb                           0.0                        0.0   
1  ionEURCb                           0.0                        0.0   
2  ionMODEm                       50000.0                     1856.0   
3   ionOGNb                           0.0                        0.0   
4    ionOPo                           0.0                        0.0   

   prev_borrow_emissions_apr  prev_borrow_net_apr  \
0                     0.0000               0.0000   
1                     0.0000               0.0000   
2                     0.1111              -0.0101   
3                     0.0000               0.0000   
4                     0.0000               0.0000   

   prev_supply_emissions_tokens  prev_supply_emissions_usd  \
0                       25000.0                      928.0   
1                       50000.0                     1856.0   
2      

<ipython-input-49-70d2fe5c6f98>:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  names_df['chain'] = names_df['chain'].replace('', np.nan).fillna(method='ffill')


In [ ]:
emissions_df.head()

,Vault,prev_borrow_emissions_tokens,prev_borrow_emissions_usd,prev_borrow_emissions_apr,prev_borrow_net_apr,prev_supply_emissions_tokens,prev_supply_emissions_usd,prev_supply_emissions_apr,prev_supply_net_apr,curr_borrow_emissions_tokens,curr_borrow_emissions_usd,curr_borrow_emissions_apr,curr_borrow_net_apr,curr_supply_emissions_tokens,curr_supply_emissions_usd,curr_supply_emissions_apr,curr_supply_net_apr,chain
0,ionAEROb,0.0,0.0,0.0000,0.0000,25000.0,928.0,0.06,0.14,0.0,0.0,0.0,-0.1387,10000.0,371.0,0.02,0.107,base
1,ionEURCb,0.0,0.0,0.0000,0.0000,50000.0,1856.0,0.28,0.33,0.0,0.0,0.0,-0.0721,15000.0,557.0,0.09,0.129,base
2,ionMODEm,50000.0,1856.0,0.1111,-0.0101,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.000,mode
3,ionOGNb,0.0,0.0,0.0000,0.0000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.000,base
4,ionOPo,0.0,0.0,0.0000,0.0000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,-0.0032,0.0,0.0,0.00,0.000,optimism


## Regression on All Vaults, Normalized, Since 10/15

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pycoingecko import CoinGeckoAPI
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from scipy import stats

# Define analysis start date and epoch dates with UTC timezone
ANALYSIS_START = pd.Timestamp('2024-09-15', tz='UTC')
PREV_EPOCH_START = pd.Timestamp('2024-10-15', tz='UTC')
PREV_EPOCH_END = pd.Timestamp('2024-11-15', tz='UTC')
CURR_EPOCH_START = pd.Timestamp('2024-11-15', tz='UTC')
CURR_EPOCH_END = pd.Timestamp('2024-12-15', tz='UTC')

# Function to safely handle datetime conversion
def safe_convert_datetime(date_series):
    dates = pd.to_datetime(date_series)
    if dates.dt.tz is None:
        return dates.dt.tz_localize('UTC')
    else:
        return dates.dt.tz_convert('UTC')

# Convert date columns to datetime and ensure UTC timezone
deposits_copy['date'] = safe_convert_datetime(deposits_copy['date'])
borrows_copy['date'] = safe_convert_datetime(borrows_copy['date'])
protocol_tvl_copy['date'] = safe_convert_datetime(protocol_tvl_copy['date'])

# Now filter data
deposits_copy = deposits_copy[deposits_copy['date'] >= ANALYSIS_START].copy()
deposits_copy['mintedAmount_USD'] = pd.to_numeric(deposits_copy['mintedAmount_USD'], errors='coerce')

borrows_copy = borrows_copy[borrows_copy['date'] >= ANALYSIS_START].copy()
borrows_copy['total_borrowed_in_vault_usd'] = pd.to_numeric(borrows_copy['total_borrowed_in_vault_usd'], errors='coerce')

protocol_tvl_copy = protocol_tvl_copy[protocol_tvl_copy['date'] >= ANALYSIS_START].copy()
protocol_tvl_copy['TVL'] = pd.to_numeric(protocol_tvl_copy['TVL'], errors='coerce')

def get_enhanced_snapshot_data(df, start_date, end_date, value_column):
    """Get enhanced snapshot data with additional metrics"""
    df = df.copy()

    start_snapshot = df[df['date'] <= start_date]['date'].max()
    end_snapshot = df[df['date'] <= end_date]['date'].max()

    if pd.isna(start_snapshot) or pd.isna(end_snapshot):
        return pd.DataFrame()

    vault_col = 'vaultName' if 'vaultName' in df.columns else 'ionicVault'

    start_data = df[df['date'] == start_snapshot][[vault_col, value_column]].copy()
    end_data = df[df['date'] == end_snapshot][[vault_col, value_column]].copy()

    start_data = start_data.rename(columns={
        vault_col: 'Vault',
        value_column: f"{value_column}_start"
    })
    end_data = end_data.rename(columns={
        vault_col: 'Vault',
        value_column: f"{value_column}_end"
    })

    return start_data.merge(end_data, on='Vault', how='outer')

def calculate_pct_change(end_val, start_val):
    """Calculate percentage change handling edge cases"""
    try:
        end_val = float(end_val) if pd.notnull(end_val) else np.nan
        start_val = float(start_val) if pd.notnull(start_val) else np.nan

        if pd.isna(start_val) and pd.isna(end_val):
            return np.nan
        if pd.isna(start_val):
            return 100.0  # New vault
        if pd.isna(end_val):
            return -100.0  # Vault closed
        if start_val == 0:
            return 100.0 if end_val > 0 else 0.0
        return ((end_val - start_val) / abs(start_val)) * 100
    except:
        return np.nan

def normalize_changes(series):
    """Normalize extreme values using winsorization"""
    if series.empty or series.isna().all():
        return series
    lower_bound = series.quantile(0.05)
    upper_bound = series.quantile(0.95)
    return series.clip(lower=lower_bound, upper=upper_bound)

def log_transform_changes(series):
    """Apply log transformation to handle extreme values"""
    if series.empty or series.isna().all():
        return series
    min_val = series.min()
    if min_val < 0:
        series = series - min_val + 1
    return np.log1p(series)

def diagnose_regression_data(df, prefix, side):
    """Diagnose regression data for potential issues"""
    features = [f'{prefix}_{side}_emissions_usd',
                f'{prefix}_{side}_net_apr',
                'TVL_start',
                'chain_encoded']
    target = f'{prefix}_{side}_pct_change_log' if side == 'borrow' else f'{prefix}_{side}_pct_change'

    print(f"\nDiagnostic for {prefix} {side}:")
    print(f"Number of observations: {len(df)}")
    print("\nFeature correlations:")
    print(df[features].corr())
    print("\nSample of data:")
    print(df[features + [target]].head())
    print("\nDescriptive statistics:")
    print(df[features + [target]].describe())
    print("\nNull values:")
    print(df[features + [target]].isnull().sum())

def run_regression(df, target_col, features):
    """Run regression with enhanced diagnostics"""
    # Drop rows with NaN values
    df_clean = df.dropna(subset=features + [target_col])

    if len(df_clean) <= len(features) + 5:
        print(f"\nWarning: Sample size ({len(df_clean)}) too small for reliable regression")
        return None

    # Check for zero variance in features
    zero_var_features = [col for col in features if df_clean[col].std() == 0]
    if zero_var_features:
        print(f"\nWarning: Zero variance in features: {zero_var_features}")
        features = [f for f in features if f not in zero_var_features]

    X = sm.add_constant(df_clean[features])
    y = df_clean[target_col]

    # Check for perfect multicollinearity
    if np.linalg.matrix_rank(X) < X.shape[1]:
        print("\nWarning: Perfect multicollinearity detected")
        return None

    model = sm.OLS(y, X).fit()

    # Print sample size and feature info
    print(f"\nFinal sample size: {len(df_clean)}")
    print(f"Features used: {features}")

    return model

# Get snapshot data for both epochs
print("Getting snapshot data for both epochs...")

# Previous epoch snapshots
prev_supply = get_enhanced_snapshot_data(deposits_copy, PREV_EPOCH_START, PREV_EPOCH_END, 'mintedAmount_USD')
prev_borrow = get_enhanced_snapshot_data(borrows_copy, PREV_EPOCH_START, PREV_EPOCH_END, 'total_borrowed_in_vault_usd')
prev_tvl = get_enhanced_snapshot_data(protocol_tvl_copy, PREV_EPOCH_START, PREV_EPOCH_END, 'TVL')

# Current epoch snapshots
curr_supply = get_enhanced_snapshot_data(deposits_copy, CURR_EPOCH_START, CURR_EPOCH_END, 'mintedAmount_USD')
curr_borrow = get_enhanced_snapshot_data(borrows_copy, CURR_EPOCH_START, CURR_EPOCH_END, 'total_borrowed_in_vault_usd')
curr_tvl = get_enhanced_snapshot_data(protocol_tvl_copy, CURR_EPOCH_START, CURR_EPOCH_END, 'TVL')

# Initialize analysis DataFrame
analysis_df = emissions_df.copy()

# Process each epoch
for prefix, supply_data, borrow_data, tvl_data in [
    ('prev', prev_supply, prev_borrow, prev_tvl),
    ('curr', curr_supply, curr_borrow, curr_tvl)
]:
    print(f"\nProcessing {prefix} epoch changes")

    # Process supply changes
    if not supply_data.empty:
        supply_changes = supply_data.copy()
        supply_changes['pct_change'] = supply_changes.apply(
            lambda x: calculate_pct_change(
                x[f'mintedAmount_USD_end'],
                x[f'mintedAmount_USD_start']
            ), axis=1
        )

        # Merge with analysis_df
        analysis_df = analysis_df.merge(
            supply_changes[['Vault', 'pct_change']],
            on='Vault',
            how='left'
        )
        analysis_df[f'{prefix}_supply_pct_change'] = normalize_changes(analysis_df['pct_change'])
        analysis_df = analysis_df.drop('pct_change', axis=1)

    # Process borrow changes
    if not borrow_data.empty:
        borrow_changes = borrow_data.copy()
        borrow_changes['pct_change'] = borrow_changes.apply(
            lambda x: calculate_pct_change(
                x[f'total_borrowed_in_vault_usd_end'],
                x[f'total_borrowed_in_vault_usd_start']
            ), axis=1
        )

        # Merge with analysis_df
        analysis_df = analysis_df.merge(
            borrow_changes[['Vault', 'pct_change']],
            on='Vault',
            how='left'
        )
        analysis_df[f'{prefix}_borrow_pct_change'] = normalize_changes(analysis_df['pct_change'])
        analysis_df[f'{prefix}_borrow_pct_change_log'] = log_transform_changes(analysis_df[f'{prefix}_borrow_pct_change'])
        analysis_df = analysis_df.drop('pct_change', axis=1)

# Merge TVL data
if not prev_tvl.empty:
    analysis_df = analysis_df.merge(
        prev_tvl[['Vault', 'TVL_start']],
        on='Vault',
        how='left'
    )

# Encode chain information
le = LabelEncoder()
analysis_df['chain_encoded'] = le.fit_transform(analysis_df['chain'].fillna('unknown'))
for prefix in ['prev', 'curr']:
    print(f"\n{prefix.upper()} EPOCH REGRESSION ANALYSIS")
    print("=" * 50)

    # Define features
    supply_features = [
        f'{prefix}_supply_emissions_usd',
        f'{prefix}_supply_net_apr',
        'TVL_start',
        'chain_encoded'
    ]

    borrow_features = [
        f'{prefix}_borrow_emissions_usd',
        f'{prefix}_borrow_net_apr',
        'TVL_start',
        'chain_encoded'
    ]

    # Supply regression
    supply_vaults = analysis_df[analysis_df[f'{prefix}_supply_emissions_usd'] > 0].copy()
    supply_model = run_regression(
        supply_vaults,
        f'{prefix}_supply_pct_change',
        supply_features
    )

    if supply_model:
        print("\nSupply Impact Regression Results:")
        print("--------------------------------")
        print(supply_model.summary().tables[1])
        print(f"R-squared: {supply_model.rsquared:.4f}")
        print(f"Number of observations: {supply_model.nobs}")

    # Borrow regression
    borrow_vaults = analysis_df[analysis_df[f'{prefix}_borrow_emissions_usd'] > 0].copy()
    borrow_model = run_regression(
        borrow_vaults,
        f'{prefix}_borrow_pct_change_log',
        borrow_features
    )

    if borrow_model:
        print("\nBorrow Impact Regression Results (Log-transformed):")
        print("------------------------------------------------")
        print(borrow_model.summary().tables[1])
        print(f"R-squared: {borrow_model.rsquared:.4f}")
        print(f"Number of observations: {borrow_model.nobs}")
    else:
        print("\nInsufficient data for borrow regression")

Getting snapshot data for both epochs...

Processing prev epoch changes

Processing curr epoch changes

PREV EPOCH REGRESSION ANALYSIS

Final sample size: 23
Features used: ['prev_supply_emissions_usd', 'prev_supply_net_apr', 'TVL_start', 'chain_encoded']

Supply Impact Regression Results:
--------------------------------
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                       497.4067    439.377      1.132      0.272    -425.690    1420.503
prev_supply_emissions_usd    -0.0316      0.278     -0.114      0.911      -0.616       0.552
prev_supply_net_apr         721.2397   1605.125      0.449      0.659   -2651.004    4093.483
TVL_start                     0.0003      0.000      2.502      0.022    4.07e-05       0.000
chain_encoded              -285.3547    308.258     -0.926      0.367    -932.981     362.272
R-squared: 0.2657


/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  res = hypotest_fun_out(*samples, **kwds)


In [ ]:
# Prepare combined regression data
supply_vaults = analysis_df[
    (analysis_df['prev_supply_emissions_usd'] > 0) |
    (analysis_df['curr_supply_emissions_usd'] > 0)
].copy()

borrow_vaults = analysis_df[
    (analysis_df['prev_borrow_emissions_usd'] > 0) |
    (analysis_df['curr_borrow_emissions_usd'] > 0)
].copy()

# Define combined features
supply_features = [
    'prev_supply_emissions_usd',
    'curr_supply_emissions_usd',
    'prev_supply_net_apr',
    'curr_supply_net_apr',
    'TVL_start',
    'chain_encoded'
]

borrow_features = [
    'prev_borrow_emissions_usd',
    'curr_borrow_emissions_usd',
    'prev_borrow_net_apr',
    'curr_borrow_net_apr',
    'TVL_start',
    'chain_encoded'
]

# Run combined supply regression
print("\nCOMBINED EPOCHS SUPPLY REGRESSION")
print("=" * 50)

supply_model = run_regression(
    supply_vaults,
    'curr_supply_pct_change',  # Using current epoch changes as target
    supply_features
)

if supply_model:
    print("\nSupply Impact Regression Results:")
    print("--------------------------------")
    print(supply_model.summary().tables[1])
    print(f"R-squared: {supply_model.rsquared:.4f}")
    print(f"Number of observations: {supply_model.nobs}")

# Run combined borrow regression
print("\nCOMBINED EPOCHS BORROW REGRESSION")
print("=" * 50)

borrow_model = run_regression(
    borrow_vaults,
    'curr_borrow_pct_change_log',  # Using current epoch changes as target
    borrow_features
)

if borrow_model:
    print("\nBorrow Impact Regression Results (Log-transformed):")
    print("------------------------------------------------")
    print(borrow_model.summary().tables[1])
    print(f"R-squared: {borrow_model.rsquared:.4f}")
    print(f"Number of observations: {borrow_model.nobs}")


COMBINED EPOCHS SUPPLY REGRESSION

Final sample size: 25
Features used: ['prev_supply_emissions_usd', 'curr_supply_emissions_usd', 'prev_supply_net_apr', 'curr_supply_net_apr', 'TVL_start', 'chain_encoded']

Supply Impact Regression Results:
--------------------------------
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                       550.4017   1325.247      0.415      0.683   -2233.839    3334.643
prev_supply_emissions_usd    -0.2017      0.849     -0.238      0.815      -1.985       1.582
curr_supply_emissions_usd    -0.1841      0.978     -0.188      0.853      -2.239       1.871
prev_supply_net_apr        5504.3719   6373.933      0.864      0.399   -7886.764    1.89e+04
curr_supply_net_apr        6301.3055    1.2e+04      0.525      0.606   -1.89e+04    3.15e+04
TVL_start                  8.242e-05      0.000      0.281      0.